In [ ]:
# Task
#Done add predict for all player_id
#Done restart conda

# add K-means
# Done add difference in 2 recent weeks
# add knee chart
# Done add evaluation metrics
# other methods of distance


# correct_answer_count is wrong surely (see describe)


# initializing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import seaborn as sns
import ipyvolume as ipv
from scipy.cluster.hierarchy import linkage,dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy import stats
import math
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

warnings.filterwarnings("ignore")

step_to_change = 1 

In [ ]:
features_catg = [
    # lifetime
    ['lifetime_recent','lifetime'],
    # games played
    ['recent_per_total_games','games_per_lifetime_recent','games_per_lifetime','total_game_recent', 'total_game','games_played_recent','games_played'],
    # won
    [ 'win_per_games_recent', 'win_per_games','won_count_recent','won_count'],
    # delete 
    ['coins_on_hint_delete_wrong_recent','coins_on_hint_delete_wrong', 'times_spent_hint_delete_wrong_recent',     'times_spent_hint_delete_wrong'],
    # again
    ['times_spent_hint_answer_again','coins_on_hint_answer_again_recent','coins_on_hint_answer_again', 'times_spent_hint_answer_again_recent'],
    # unlimited 
    ['unlimited_play_per_games_recent','unlimited_play_per_spent_recent', 'unlimited_play_per_games','times_spent_unlimited_games','coins_spent_on_unlimited_games_recent','coins_spent_on_unlimited_games',  'times_spent_unlimited_games_recent'],
    # avatars
    ['change_avatar_per_games_recent','times_change_avatar_per_spent_recent','coins_spent_on_avatars_per_coins_spent_recent','times_changing_avatars','change_avatar_per_games','times_change_avatar_per_spent','coins_spent_on_avatars_recent','coins_spent_on_avatars','times_changing_avatars_recent'],
    # freeze 
    ['coins_on_hint_freeze_time_recent','coins_on_hint_freeze_time','times_spent_hint_freeze_time','times_spent_hint_freeze_time_recent'],
    # booster
    ['booster_per_games_recent','booster_per_games','times_spent_booster_package','coins_spent_on_booster_package_recent','coins_spent_on_booster_package','times_spent_booster_package_recent'],
    # percentage
    ['times_hint_answer_percentage','coins_on_hint_answer_per_recent','coins_on_hint_answer_per', 'times_hint_answer_percentage_recent'],
    # coin
    ['adv_coin_per_games_recent','times_viewing_adv_free_coin','times_viewing_adv_free_coin_recent'],
    # extra
    ['adv_extra_game_per_games_recent','times_viewing_adv_extra_game','times_viewing_adv_extra_game_recent'],
    # perfect
    ['perfect_games_recent','perfect_games'],
    # won expire
    ['win_by_expir_per_games_recent', 'win_by_expir_per_games','won_count_with_expiration','won_count_with_expiration_recent'],
    # lose expire
    ['lose_by_expir_per_games_recent','lose_by_expir_per_games','lost_count_with_expiration','lost_count_with_expiration_recent'],
    # cups
    ['tournament_cups','tournament_cups_recent'],
    # tournaments
    ['tournament_per_games_recent','tournaments_played','tournaments_played_recent'],
    # invitation
    ['number_of_invitation','number_of_invitation_recent'],
    # correct
    ['correct_answer_per_games_recent','correct_answer_per_games','correct_answer_count','correct_answer_count_recent'],
    # spent
    ['total_coins_spent_recent','total_times_spent','total_times_spent_recent','total_coins_spent'],
    # guide
    ['guide_per_games','guide_per_spent_recent',  'guide_per_games_recent'],
    # 
    ['expir_tech_or_laziness_recent']
]



In [ ]:
def feature_eval(n,features,data,features_catg):
    modeling_feed=data[features]
    #modeling
    clustering = GaussianMixture(n_components=n).fit_predict(modeling_feed)
    modeling_feed['cat']=clustering
    #evaluation
    modeling_feed_smp = modeling_feed.sample(frac=0.1, random_state=1)
    modeling_feed_smp2 = modeling_feed.sample(frac=0.02, random_state=1)
    score_davies_bouldin = davies_bouldin_score(modeling_feed_smp[features],modeling_feed_smp.cat)
    my_score = my_score_calc_v2(modeling_feed_smp,features_catg,10)
    score_silhouette = 0
    if score_davies_bouldin<1.5:
        print('+',end='')
        score_silhouette = silhouette_score(modeling_feed_smp[features], modeling_feed_smp.cat)
        print('*',end='')
    elif score_davies_bouldin<3:
        print('++',end='')
        score_silhouette = silhouette_score(modeling_feed_smp2[features], modeling_feed_smp2.cat)
        print('**',end='')
    max_cluster = max(modeling_feed.cat.value_counts()/len(modeling_feed)*100)
    min_cluster = min(modeling_feed.cat.value_counts()/len(modeling_feed)*100)
    #more accurate evaluation
    
    if (score_silhouette>0.8#best_score 
        and max_cluster<200/i 
        and min_cluster>(2**((8-i)-1)+1)): 
        score_davies_bouldin = davies_bouldin_score(modeling_feed[features],modeling_feed.cat)
        score_silhouette = silhouette_score(modeling_feed[features], modeling_feed.cat)
        
        print({'c':c,
              'sil':score_silhouette,
              'dav':score_davies_bouldin,
              'max_c':max_cluster,
              'min_c':min_cluster,
              'n_cluster':i})
        
    return score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster

In [ ]:
def features_evaluations(min_n_cluster,max_n_cluster,base_features,available_features,data,features_catg):
    import pickle
    import os.path
    result_columns = ['sign','added_feature','n_cluster','score_silhouette','score_davies_bouldin','my_score','max_cluster','min_cluster','all_features','new_run']
 
    if os.path.isfile('Data/processed_list.pickle'):
        with open('Data/processed_list.pickle', 'rb') as f:
            processed_list = pickle.load(f)
        processed_list.sign='other_results'
    else:
        processed_list = pd.DataFrame(columns=result_columns)
    processed_list.reset_index(inplace=True,drop=True)
    clusters_range = range(min_n_cluster,max_n_cluster+1)
    #base_features=[ 'coins_spent_on_avatars_per_coins_spent','unlimited_play_per_spent','tournament_per_games','guide_per_games']
    processed_list['new_run']=0
    print('start calculating')

    
    
    
    feature_list=base_features
    for i in clusters_range:
        if (len(processed_list[(processed_list.n_cluster==i)&(processed_list.all_features.apply(set)==set(base_features))])==0
            and len(feature_list)>0):
            print('base',end='')
            print(i,end='$')
            score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster = feature_eval(i,feature_list,data,features_catg)
                        
            new_row = ['base_features','',i,score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster,feature_list,1]
            processed_list=processed_list.append(pd.DataFrame(new_row,columns=result_columns), ignore_index=True)

        else:
            processed_list.loc[(processed_list.n_cluster==i)&(processed_list.all_features.apply(set)==set(feature_list)),['new_run','sign','added_feature']]=[1,'base_features','']
    
    
    
    
    
    for c in available_features:
        if not (is_related_list(c,base_features,features_catg) 
                or c in base_features):
            feature_list=base_features+[c]
            for i in clusters_range:
                if len(processed_list[(processed_list.n_cluster==i)&(processed_list.all_features.apply(set)==set(feature_list))])==0:
                    print(c,end='')
                    print(i,end='.')
                    score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster = feature_eval(i,feature_list,data,features_catg)
                    
                    
                    new_row = ['+',c,i,score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster,feature_list,1]
                    processed_list=processed_list.append(pd.DataFrame(new_row,columns=result_columns), ignore_index=True)
                    
                    
            else:
                processed_list.loc[(processed_list.n_cluster==i)&(processed_list.all_features.apply(set)==set(feature_list)),['new_run','sign','added_feature']]=[1,'+',c]
    
    
    
    
    
    
    for c in base_features:
        #print('%',c,'%')
        feature_list=list(set(base_features)-set([c]))
        for i in clusters_range:
            try:
                print(len(processed_list[(processed_list.n_cluster==i)&((processed_list.all_features.apply(set)==set(feature_list)))]))
            except: 
                print('error 54')
            
            
            
            if (len(processed_list[(processed_list.n_cluster==i)&((processed_list.all_features.apply(set)==set(feature_list)))])==0
                and len(feature_list)>0):
                print(c,end='')
                print(i,end=',')
                score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster = feature_eval(i,feature_list,data,features_catg)
                
                
                
                new_row = ['-',c,i,score_silhouette,score_davies_bouldin,my_score,max_cluster,min_cluster,feature_list,1]
                processed_list=processed_list.append(pd.DataFrame(new_row,columns=result_columns), ignore_index=True)

        else:
            processed_list.loc[(processed_list.n_cluster==i)&(processed_list.all_features.apply(set)==set(feature_list)),['new_run','sign','added_feature']]=[1,'-',c]
                
    
    
    
    
    
    
    with open('Data/processed_list.pickle', 'wb') as f:
        pickle.dump(processed_list, f)

    return processed_list.loc[processed_list.new_run==1,['sign','added_feature','n_cluster','score_silhouette','score_davies_bouldin','my_score','max_cluster','min_cluster','all_features']],processed_list


In [ ]:
def show_result(data,n_cluster,features,fraction,true_data):
    df=data.copy()
    tdf = true_data.copy()
    
    clustering = GaussianMixture(n_components=n_cluster).fit_predict(df[features])
    df['cat']=clustering
    tdf['cat'] = clustering
    
    
    df = df.sample(frac=fraction,random_state=1)
    tdf = tdf.sample(frac=fraction,random_state=1)
    
    g1 = sns.countplot(x="cat",data=df)
    print(df.cat.value_counts()/len(df)*100)
    plt.show()

    
    
    rel = pd.DataFrame(columns=['col','score','p_value'])
    cat_list = df.cat.unique()
    for c in (set(df.columns)-set(['cat'])):
        arg=[]
        for ct in cat_list:
            arg+=[df[df.cat==ct][c]]
        s,p=stats.f_oneway(*arg)
        rel.loc[len(rel)]=[c,abs(s),p]
    print(rel.sort_values(by=['p_value','score'],ascending=[True,False]).head(10))
    affected_features = rel.sort_values(by=['p_value','score'],ascending=[True,False]).head(10).col.tolist()
    
    
    
    print('######### with transformed data ##########')
    coln = len(df.columns)-1
    fig,axes = plt.subplots(1,3,figsize=(16,2))
    i=0
    for c in df.columns:
        if c in set(features+affected_features):
            sns.factorplot( y= c,x = "cat",data = df, kind="violin",ax=axes[i%3],scale="width")
            i+=1
            plt.close(2)
            if i%3==0:
                plt.show()
                if i!=coln:
                    fig,axes = plt.subplots(1,3,figsize=(16,2))
    plt.show()                
                    
    print('######### with True data ##########')
    coln = len(tdf.columns)-1
    fig,axes = plt.subplots(1,3,figsize=(16,2))
    i=0
    for c in tdf.columns:
        if c in set(features+affected_features):
            sns.factorplot( y= c,x = "cat",data = tdf, kind="violin",ax=axes[i%3],scale="width")
            i+=1
            plt.close(2)
            if i%3==0:
                plt.show()
                if i!=coln:
                    fig,axes = plt.subplots(1,3,figsize=(16,2))

In [ ]:
def is_related_list(c,l,features_catg):
    for c2 in l:
        if is_related(c,c2,features_catg):
            return True
    return False

In [ ]:
def is_related(c1,c2,features_catg):
    for i in range(len(features_catg)):
        if c1 in features_catg[i] and c2 in features_catg[i]:
            return True
    return False

In [ ]:
'''
def correct_score(data,features,n,features_catg):
    modeling_feed=data[features]
    #modeling
    clustering = GaussianMixture(n_components=n).fit_predict(modeling_feed)
    modeling_feed['cat']=clustering
    print(features,n)
    return my_score_calc_v2(modeling_feed,features_catg,10)

all_result.my_score = all_result.apply(lambda r: correct_score(df7.sample(frac=0.2,random_state=1),r.all_features,r.n_cluster,features_catg),axis=1)
'''

In [ ]:
'''
def my_score_calc(df,features_catg,n_first):
    rel = pd.DataFrame(columns=['col','score','p_value'])
    cat_list = df.cat.unique()
    for c in (set(df.columns)-set(['cat'])):
        arg=[]
        for ct in cat_list:
            arg+=[df[df.cat==ct][c]]
        s,p=stats.f_oneway(*arg)
        rel.loc[len(rel)]=[c,abs(s),p]
    try:
        affected_features = rel.sort_values(by=['p_value','score'],ascending=[True,False]).col.tolist()
    except:
        print(rel)
    score=0
    for cl in range(len(affected_features)):
        clp=cl-1
        while clp>=0:
            if is_related(affected_features[cl],affected_features[clp],features_catg):
                break
            clp-=1
        if clp==-1:
            score += rel[rel.col==affected_features[cl]].score.iloc[0]
            n_first-=1
            if n_first==0:
                break
    return score
'''

In [ ]:
def my_score_calc_v2(df,features_catg,n_first):
    rel = pd.DataFrame(columns=['col','score','p_value'])
    cat_list = df.cat.unique()
    for c in (set(df.columns)-set(['cat'])):
        arg=[]
        for ct in cat_list:
            arg+=[df[df.cat==ct][c]]
        s,p=stats.f_oneway(*arg)
        rel.loc[len(rel)]=[c,abs(s),p]
    try:
        affected_features = rel.sort_values(by=['p_value','score'],ascending=[True,False]).col.tolist()
    except:
        print('Error',rel)
    score=0
    cl=0
    inf = 0
    while n_first>0 and cl<len(affected_features):
        if not is_related_list(affected_features[cl],affected_features[:cl],features_catg):
            try:
                score += round(math.log(rel[rel.col==affected_features[cl]].score.iloc[0],10))
                score += inf * score #add max instead of infinity
                inf = 0 
            except: # score = infinity
                inf+=1
            n_first-=1
        cl+=1
    return score


In [ ]:
def save_result(data,n_cluster,features,player_id,name):
    df=data[features]
    
    clustering = GaussianMixture(n_components=n_cluster).fit_predict(df)
    df['cat']=clustering
    df['player_id'] = player_id
    filename = 'cats__'+name+'__'+str(n_cluster)+'__'+str(features)+'.csv'
    df[['player_id','cat']].to_csv(filename,index=False)
    print('Done: saving '+filename)

In [ ]:
def sil_coeff(no_clusters,data,show=False):
    # Apply your clustering algorithm of choice to the reduced data
    clustering = GaussianMixture(n_components=5).fit_predict(data)
    score = silhouette_score(data, clustering)
    '''
    clusterer_1 = KMeans(n_clusters=no_clusters, random_state=0 )
    clusterer_1.fit(data)
    
    # Predict the cluster for each data point
    preds_1 = clusterer_1.predict(data)
    
    # Calculate the mean silhouette coefficient for the number of clusters chosen
    score = silhouette_score(data, preds_1)
    '''
    if show:
        sns.factorplot( y= 'val',x = "cat",data = data, kind="violin")
        
    return score

In [ ]:
def silh(X_scaled):
    K_best=8
    cluster_centers = dict()

    for n_clusters in range(3,K_best+1,1):
        fig, ax1 = plt.subplots(1, 1)
        #fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
        fig.set_size_inches(25, 7)
        ax1.set_xlim([-0.1, 1])
        ax1.set_ylim([0, len(X_scaled) + (n_clusters + 1) * 10])

        clusterer = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10,max_iter=300, tol=1e-04, random_state=101)
        cluster_labels = clusterer.fit_predict(X_scaled)

        silhouette_avg = silhouette_score(X = X_scaled, labels = cluster_labels)
        cluster_centers.update({n_clusters :{'cluster_center':clusterer.cluster_centers_,
                                             'silhouette_score':silhouette_avg,
                                             'labels':cluster_labels}
                               })

        sample_silhouette_values = silhouette_samples(X = X_scaled, labels = cluster_labels)
        y_lower = 10
        for i in range(n_clusters):
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.Spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
        ax1.set_yticks([])
        ax1.set_xticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
        colors = cm.Spectral(cluster_labels.astype(float) / n_clusters)
        plt.show()

## reading data

In [ ]:
newdata=pd.read_csv('Data/2019_03_29.csv', error_bad_lines=False)

newdata.shape

In [ ]:
pastdata=pd.read_csv('Data/2019_03_14.csv', error_bad_lines=False)

pastdata.shape

In [ ]:
data = newdata.merge(right=pastdata, how='inner',left_on='player_id',right_on='player_id', sort=False,suffixes=('', '_past'))

data.shape

# cleaning


### number_of_invitation (-1 to 0)

In [ ]:
def correct_minues_one(newdata):
    newdata.loc[newdata.number_of_invitation<0,'number_of_invitation']=0
    newdata.loc[newdata.correct_answer_count<0,'correct_answer_count']=0
    print(newdata.shape)
    return newdata

In [ ]:
clean_data = correct_minues_one(data)

# reducing data

## defining steps

### delete inactive users

In [ ]:
def delete_inactive_users_v3(data):
    df=data.copy()
    
    df = df[df.games_played>10]
    df = df[df.games_played>df.games_played_past]
    
    print(df.shape)
    return df

In [ ]:
'''
def delete_inactive_users_v2(data):
    df=data.copy()
    df.updated_at = df.updated_at.apply(lambda x:datetime.strptime(x[:10],'%Y-%m-%d'))
    df = df[df.updated_at>=datetime.strptime('2019-03-01','%Y-%m-%d')]
    df = df[df.games_played>10]
    print(df.shape)
    return df
'''

In [ ]:
'''
def delete_inactive_users(newdata,olddata):
    previous_games_played = olddata[['player_id','games_played','created_at']]
    previous_games_played.columns = ['player_id','previous_games_played','previous_created_at']

    newdata_withprevious_games_count = newdata.merge(right=previous_games_played, how='inner',left_on='player_id',right_on='player_id', sort=False)

    active_users = newdata_withprevious_games_count[newdata_withprevious_games_count.games_played>newdata_withprevious_games_count.previous_games_played]
    active_users = active_users[active_users.games_played>5]
    print('Done: delete_inactive_users')
    return active_users
'''

### delete non-buyer users

In [ ]:
def none_buyer(data):
    df=data.copy()
    result = df[df.purchased_times==0]
    print(result.shape)
    return result

### calculate lifetime

In [ ]:
def add_lifetime_v2(data):
    df=data.copy()
    df.qok_created_at = df.qok_created_at.apply(lambda x:datetime.strptime(x[:10],'%Y-%m-%d'))
    df['lifetime'] = (max(df.qok_created_at) - df.qok_created_at).apply(lambda x:x.days)+1
    df['lifetime_recent'] = 15
    print(df.shape)
    return df

## pipline

In [ ]:
dr1 = delete_inactive_users_v3(clean_data)

In [ ]:
dr2 = none_buyer(dr1)

In [ ]:
dr3 = add_lifetime_v2(dr2)

In [ ]:
reduced_data = dr3.copy()

# feature_engineering

## defining steps

### define recents

In [ ]:
def define_recents(data):
    df=data.copy()
    for c in df.columns:
        if c not in ['player_id','updated_at','created_at','account_id','qok_created_at'] and\
           c+'_past' in df.columns:
            df[c+'_recent'] = df[c]-df[c+'_past']
    print(df.shape)
    return df

### total coin & coin

In [ ]:
def cal_total_times_n_coins(data):
    df=data.copy()
    for c in ['','_recent']:
        df['total_times_spent'+c] =df['times_spent_hint_delete_wrong'+c]+\
                                        df['times_spent_hint_answer_again'+c]+\
                                        df['times_spent_unlimited_games'+c]+\
                                        df['times_changing_avatars'+c]+\
                                        df['times_spent_hint_freeze_time'+c]+\
                                        df['times_spent_booster_package'+c]+\
                                        df['times_hint_answer_percentage'+c]+\
                                        df['tournaments_played'+c] + 1

        df['total_coins_spent'+c]=df['coins_on_hint_delete_wrong'+c]+\
                                        df['coins_on_hint_answer_again'+c]+\
                                        df['coins_spent_on_unlimited_games'+c]+\
                                        df['coins_spent_on_avatars'+c]+\
                                        df['coins_on_hint_freeze_time'+c]+\
                                        df['coins_spent_on_booster_package'+c]+\
                                        df['coins_on_hint_answer_per'+c]+\
                                        (df['tournaments_played'+c]*15)+\
                                        (df['lost_count_with_expiration'+c]*50) + 1


    print(df.shape)
    return df

### round features

In [ ]:
def round_some_features(data):
    df=data.copy()
    df.games_played=round(df.games_played,-1)
    df.games_played_recent = round(df.games_played_recent,-1)
    print(df.shape)
    return df

### remove unrelated col 

In [ ]:
def remove_unrelated_column_to_project(data):
    df=data.copy()
    col_list = [
        'lifetime',
       'games_played',  
        'won_count', 

        'coins_on_hint_delete_wrong',
        'coins_on_hint_answer_again',
        'coins_spent_on_unlimited_games',
        'coins_spent_on_avatars',
        'coins_on_hint_freeze_time',
        'coins_spent_on_booster_package',
        'coins_on_hint_answer_per',
                                 
        'times_spent_hint_freeze_time',
        'times_spent_hint_delete_wrong',
        'times_spent_hint_answer_again',
        'times_hint_answer_percentage',
        'times_spent_booster_package',
                                 
       'times_spent_unlimited_games',
       'times_changing_avatars',
                                 
        'times_viewing_adv_free_coin', 
        'times_viewing_adv_extra_game',

        'perfect_games', 
        'won_count_with_expiration',
        'lost_count_with_expiration',
                                 
        'tournament_cups',
        'tournaments_played', 
        'number_of_invitation',
        'correct_answer_count']
    col_list += [s+'_recent' if s+'_recent' in df.columns else s+'_past' for s in col_list]
    result = df[col_list]
    print(result.shape)
    return result

### derived features

In [ ]:
def create_derived_features(data):
    df=data.copy()
    for c in ['','_recent']:
        df['total_game'+c] = (df['games_played'+c]+df['tournaments_played'+c])
        df['coins_spent_on_avatars_per_coins_spent'+c]=df['coins_spent_on_avatars'+c]/df['total_coins_spent'+c]

        df['unlimited_play_per_spent'+c] = df['times_spent_unlimited_games'+c] / df['total_times_spent'+c]
        df['times_change_avatar_per_spent'+c] = df['times_changing_avatars'+c] / df['total_times_spent'+c]
        df['guide_per_spent'+c] = (df['times_spent_hint_answer_again'+c]+
                                                df['times_spent_hint_delete_wrong'+c]+
                                                df['times_spent_hint_freeze_time'+c]+
                                                df['times_hint_answer_percentage'+c])/df['total_times_spent'+c]


        df['booster_per_games'+c] = df['times_spent_booster_package'+c] / df['total_game'+c]
        df['unlimited_play_per_games'+c] = df['times_spent_unlimited_games'+c] / df['total_game'+c]
        df['change_avatar_per_games'+c] = df['times_changing_avatars'+c] / df['total_game'+c]
        df['guide_per_games'+c] = (df['times_spent_hint_answer_again'+c]+
                                                df['times_spent_hint_delete_wrong'+c]+
                                                df['times_spent_hint_freeze_time'+c]+
                                                df['times_hint_answer_percentage'+c])/df['total_game'+c]

        df['win_per_games'+c] = df['won_count'+c] / df['total_game'+c]

        df['adv_coin_per_games'+c] = df['times_viewing_adv_free_coin'+c] / df['total_game'+c]
        df['adv_extra_game_per_games'+c] = df['times_viewing_adv_extra_game'+c] / df['total_game'+c]

        df['games_per_lifetime'+c] = df['total_game'+c] / df['lifetime'+c]

        df['tournament_per_games'+c] = df['tournaments_played'+c] / df['total_game'+c]

        df['correct_answer_per_games'+c] = df['correct_answer_count'+c] / df['total_game'+c]

        df['win_by_expir_per_games'+c] = df['won_count_with_expiration'+c] / df['games_played'+c]
        df['lose_by_expir_per_games'+c] = df['lost_count_with_expiration'+c] / df['games_played'+c]

        df['expir_tech_or_laziness'+c] = df['won_count_with_expiration'+c] / (df['lost_count_with_expiration'+c]+1)

        

        df['save'+c] = (df['won_count'+c]*20+\
                                      df['times_viewing_adv_free_coin'+c]*30+\
                                      df['number_of_invitation'+c]*200+
                                      df['tournament_cups'+c]*70)\
                                    -df['total_coins_spent'+c]
        
        
    df['recent_per_total_games'] = df['games_played_recent']/df['total_game']
    
    print(df.shape)
    return df

### remove outliers & extra features

In [ ]:
def fix_outliers(data):
    df=data.copy()
    for c in df.columns: 
        df.loc[df[c]>=df[c].quantile(.9995),c] = df[c].quantile(.9995)
        df.loc[df[c]<=df[c].quantile(.0005),c] = df[c].quantile(.0005)
    return df

In [ ]:
def features_selection(data,selected_features):
    df=data.copy()
    per_list=[]
    recent_list=[]
    others_list=[]
    for c in df.columns:
        if c not in selected_features:
            if 'recent' in c:
                recent_list+=[c]
            elif 'per' in c:
                per_list+=[c]
            else:
                others_list+=[c]
    print(df[main_features].shape)
    print('recent:',recent_list,'\n')
    print('per:',per_list,'\n')
    print('others',others_list)
    return df[selected_features]

### transform

In [ ]:
def transform(data):
    result = data.copy()
    for c in result.columns:
        if c not in ['win_per_games','player_id','win_per_games_recent','player_id_past']:
            x=result[c]-min(result[c])+1
            xt, _ = stats.boxcox(x)
            result[c] = xt
            result[c] = result[c].apply(lambda x: round(x,2))
    print(result.shape)
    return result

### sampling

In [ ]:
def sampling(trans_data,true_data,sample_fraction):
    idx = pd.DataFrame({'i':true_data.index})
    smp = idx.sample(frac=sample_fraction, random_state=1)
    idx_smp = smp['i'].tolist()
    true_data = true_data.ix[idx_smp]
    trans_data = trans_data.ix[idx_smp]
    #clustering_data_samp = data.sample(frac=sample_fraction, random_state=1)
    print(trans_data.shape,true_data.shape)
    return trans_data,true_data

### PCA

In [ ]:

def pca_results(good_data, pca,ax):
    '''
    Create a DataFrame of the PCA results
    Includes dimension feature weights and explained variance
    Visualizes the PCA results
    '''

    # Dimension indexing
    dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = list(good_data.keys()))
    components.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
    variance_ratios.index = dimensions

    # Create a bar plot visualization
    #fig, ax = plt.subplots(figsize = (14,8))

    # Plot the feature weights as a function of the components
    components.plot(ax = ax, kind = 'bar');
    ax.set_ylabel("Feature Weights")
    ax.set_xticklabels(dimensions, rotation=0)


    # Display the explained variance ratios
    for i, ev in enumerate(pca.explained_variance_ratio_):
        ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

    # Return a concatenated DataFrame
    return pd.concat([variance_ratios, components], axis = 1)


In [ ]:
def pca(data,n_comp):
    fig, ax = plt.subplots(figsize=(6+n_comp*3,2+n_comp*3)) 
    from sklearn.decomposition import PCA
    pca_feed=data.copy()
    # Apply PCA by fitting the good data with the same number of dimensions as features
    pca = PCA(n_components = n_comp, random_state=0)
    pca.fit(pca_feed)

    # Transform log_samples using the PCA fit above
    pca_t = pca.transform(pca_feed)
    print("Explained Variance Ratio => {}\n".format(pca.explained_variance_ratio_))
    print("Explained Variance Ratio(csum) => {}\n".format(pca.explained_variance_ratio_.cumsum()))

    # Generate PCA results plot
    dim_var = pca_results(pca_feed, pca,ax)
    dv = dim_var.pivot_table(values=list(set(dim_var.columns)-set(['Explained Variance'])),columns=list(dim_var.index))
    dv2 = dv.copy()
    for d in dv2.columns:
        dv2[d] = dv2[d].apply(abs)
        dv2.sort_values(by=d,inplace=True,ascending=False)
        v1 = set(dv2[dv2[d]>0.25].index)
        v2 = set(dv2.index[[0,1]])
        #print(d,(v1|v2))
        print(d,end='\t')
        for v in list(v2|v1):
            print(v,dv.loc[v,d],end='\t')
        print('')

    print(pca_t.shape)
    return pca_t,pca

## feature engineering pipline

In [ ]:
df1 = define_recents(reduced_data)

In [ ]:
df2 = remove_unrelated_column_to_project(df1)

In [ ]:
df3 = cal_total_times_n_coins(df2)

In [ ]:
df4 = create_derived_features(df3)

In [ ]:
#df5 = round_some_features(df4)
df5 = df4.copy()

In [ ]:
df6 = fix_outliers(df5)

In [ ]:
df7 = transform(df6)

In [ ]:
main_features=[ 
    
    #'win_by_expir_per_games',
    'expir_tech_or_laziness',
    #'win_per_games',
    'guide_per_spent',

    'coins_spent_on_avatars_per_coins_spent',
    
    'unlimited_play_per_spent',
    'tournament_per_games',
    'adv_extra_game_per_games', 
    
    'adv_coin_per_games' 
    #stable
    #'recent_per_total_games'
]
df8 = features_selection(df7,main_features) #main features , all features

In [ ]:
data_trans = df7.copy()
true_data = df6.copy()

# diminish data for modeling

## steps

In [ ]:
def do_pca(data_trans,true_data):
    if data_trans.isnull().values.any():
        data_trans.info()
        return
    dd1,true_data_smp = sampling(data_trans,true_data,0.05)
    dd2, pca_fitted = pca(dd1,10)
    dd3 = dd2[:,range(0,10)]
    return pca_fitted.transform(data_trans)

In [ ]:
def do_kmeans(data):
    kmeans = KMeans(n_clusters = 200, init = 'k-means++', max_iter = 100, n_init = 10, random_state = 0)
    kmeans.fit(data.sample(frac=0.15, random_state=1))

    _, counts = np.unique(kmeans.labels_, return_counts=True)
    print(stats.describe(counts))
    df = kmeans.predict(data)
    return kmeans.cluster_centers_, df

## pipline

In [ ]:
active_PCA =False
if active_PCA:
    print('PCA: ON')
    dd0 = do_pca(data_trans,true_data)
else:
    print('PCA: OFF')
    dd0 = data_trans.copy()

dd1 = pd.DataFrame(data=dd0)

In [ ]:
modeling_feed = dd3.copy()

In [ ]:
_=[
    # lifetime
    ['lifetime_recent','lifetime'],
    # games played
    ['recent_per_total_games','games_per_lifetime_recent','games_per_lifetime','total_game_recent', 'total_game','games_played_recent','games_played'],
    # won
    [ 'win_per_games_recent', 'win_per_games','won_count_recent','won_count'],
    # delete 
    ['coins_on_hint_delete_wrong_recent','coins_on_hint_delete_wrong', 'times_spent_hint_delete_wrong_recent',     'times_spent_hint_delete_wrong'],
    # again
    ['times_spent_hint_answer_again','coins_on_hint_answer_again_recent','coins_on_hint_answer_again', 'times_spent_hint_answer_again_recent'],
    # unlimited 
    ['unlimited_play_per_games_recent','unlimited_play_per_spent_recent', 'unlimited_play_per_games','times_spent_unlimited_games','coins_spent_on_unlimited_games_recent','coins_spent_on_unlimited_games',  'times_spent_unlimited_games_recent'],
    # avatars
    ['change_avatar_per_games_recent','times_change_avatar_per_spent_recent','coins_spent_on_avatars_per_coins_spent_recent','times_changing_avatars','change_avatar_per_games','times_change_avatar_per_spent','coins_spent_on_avatars_recent','coins_spent_on_avatars','times_changing_avatars_recent'],
    # freeze 
    ['coins_on_hint_freeze_time_recent','coins_on_hint_freeze_time','times_spent_hint_freeze_time','times_spent_hint_freeze_time_recent'],
    # booster
    ['booster_per_games_recent','booster_per_games','times_spent_booster_package','coins_spent_on_booster_package_recent','coins_spent_on_booster_package','times_spent_booster_package_recent'],
    # percentage
    ['times_hint_answer_percentage','coins_on_hint_answer_per_recent','coins_on_hint_answer_per', 'times_hint_answer_percentage_recent'],
    # coin
    ['adv_coin_per_games_recent','times_viewing_adv_free_coin','times_viewing_adv_free_coin_recent'],
    # extra
    ['adv_extra_game_per_games_recent','times_viewing_adv_extra_game','times_viewing_adv_extra_game_recent'],
    # perfect
    ['perfect_games_recent','perfect_games'],
    # won expire
    ['win_by_expir_per_games_recent', 'win_by_expir_per_games','won_count_with_expiration','won_count_with_expiration_recent'],
    # lose expire
    ['lose_by_expir_per_games_recent','lose_by_expir_per_games','lost_count_with_expiration','lost_count_with_expiration_recent'],
    # cups
    ['tournament_cups','tournament_cups_recent'],
    # tournaments
    ['tournament_per_games_recent','tournaments_played','tournaments_played_recent'],
    # invitation
    ['number_of_invitation','number_of_invitation_recent'],
    # correct
    ['correct_answer_per_games_recent','correct_answer_per_games','correct_answer_count','correct_answer_count_recent'],
    # spent
    ['total_coins_spent_recent','total_times_spent','total_times_spent_recent','total_coins_spent'],
    # guide
    ['guide_per_games','guide_per_spent_recent',  'guide_per_games_recent'],
    # 
    ['expir_tech_or_laziness_recent']
]

# modeling

In [ ]:
#all available features
#new_list = data_trans.columns 

# without recent
new_list = ['coins_on_hint_answer_per', 'times_hint_answer_percentage', 'perfect_games', 'times_change_avatar_per_spent', 'booster_per_games', 'unlimited_play_per_games', 'change_avatar_per_games', 'guide_per_games', 'win_per_games', 'games_per_lifetime', 'correct_answer_per_games', 'win_by_expir_per_games', 'lose_by_expir_per_games','lifetime', 'games_played', 'won_count', 'coins_on_hint_delete_wrong', 'coins_on_hint_answer_again', 'coins_spent_on_unlimited_games', 'coins_spent_on_avatars', 'coins_on_hint_freeze_time', 'coins_spent_on_booster_package', 'times_spent_hint_freeze_time', 'times_spent_hint_delete_wrong', 'times_spent_hint_answer_again', 'times_spent_booster_package', 'times_spent_unlimited_games', 'times_changing_avatars', 'times_viewing_adv_free_coin', 'times_viewing_adv_extra_game', 'won_count_with_expiration', 'lost_count_with_expiration', 'tournament_cups', 'tournaments_played', 'number_of_invitation', 'correct_answer_count', 'total_times_spent', 'total_coins_spent', 'total_game', 'save']

base_features = ['coins_spent_on_avatars_per_coins_spent','coins_on_hint_freeze_time_recent']

result, all_result = \
    features_evaluations(min_n_cluster=3,
                        max_n_cluster=4,
                        base_features= base_features,
                        available_features = new_list,
                        data = data_trans,
                        features_catg = features_catg)


In [ ]:
result

In [ ]:
result[#(result.score_silhouette>0.8)& # filter on silhouette score
               (result.min_cluster>1)& # size of smallest cluster in percentage
               (result.max_cluster<60)  # size of biggest cluster in percentage
              ].sort_values(by='score_silhouette',ascending=False)

In [ ]:
result.loc[129].all_features  # shows all_features column by index of row

In [ ]:
result[(result.score_silhouette>0.8)& # filter clusterings result based on silhouette score
               (result.min_cluster>(2**((8-result.n_cluster)-1)+1))& # automatic formula for filter clustering with too small cluster size
               (result.max_cluster<200/result.n_cluster)  # automatic formula for filter clustering with too big cluster size
              ].sort_values(by='score_silhouette',ascending=False) # sort results

# evaluation a model

In [ ]:
# to show result in full page : 
    # 1. press Esc to make this cell blue (vertical line of this cell)
    # 2. then press O
ft=['coins_spent_on_avatars_per_coins_spent','coins_on_hint_answer_again_recent',
    'unlimited_play_per_spent','tournament_per_games']
show_result(data = data_trans,
            n_cluster= 4,
            features= ft,
            fraction= 0.3,
            true_data= true_data )



# Deployment

In [ ]:
selected_features_for_clustering=['coins_spent_on_avatars_per_coins_spent','coins_on_hint_answer_again_recent',
    'unlimited_play_per_spent','tournament_per_games']

save_result(data = data_trans,
            n_cluster = 4,
            features = selected_features_for_clustering,
            player_id = reduced_data.player_id,
            name = 'function_test')